In [1]:
!pip install scipy

In [2]:
!pip install pandas

In [3]:
!pip install numpy

## setting params

In [4]:
#this should be right but i'm not 100%

import scipy.io
import pandas as pd
#from CellFie_stub import CellFie

mat_data = scipy.io.loadmat('dataTest.mat')
#print(mat_data.keys())

gene_data = mat_data['data'][0, 0]['gene']
#print('--gene--')
#print(type(gene_data))
#print(gene_data.shape)

tissue_data = mat_data['data'][0, 0]['Tissue']
#print('--tissue--')
#print(type(tissue_data))
#print(tissue_data.shape)

value_data = mat_data['data'][0, 0]['value']
#print('--value--')
#print(type(value_data))
#print(value_data.shape)

data_void = mat_data['data'][0, 0]
data= {
    'gene': data_void['gene'],
    'Tissue': data_void['Tissue'],
    'value': data_void['value']
}

#print(type(data))

SampleNumber=3

ref='MT_recon_2_2_entrez'

#making this a dict seems best
param={
    'ThreshType':'local',
    'LocalThresholdType':'minmaxmean',
    'percentile_or_value':'percentile',
    'percentile_low': 25,
    'percentile_high': 75
}

pathway='metabolism'

#unsure if set or list is best here, gpt says unique
sampNames=['samp1','samp2','samp3','samp4']

In [5]:
#cellfie function imports

import pandas as pd
import numpy as np
import scipy.io
import collections
from findUsedGenesLevels_all_V2 import findUsedGenesLevels_all
from selectGeneFromGPR_all_V2 import selectGeneFromGPR_all

## param checks

In [6]:
if data['value'].shape[1] != SampleNumber:
    raise ValueError('The number of samples defined is not the same as the size of the dataset')
if data['value'].shape[0] != len(data['gene']):
    raise ValueError('data.value does not have the same number of rows as data.gene')
if 'ref' not in locals() and 'ref' not in globals():
    raise ValueError('The reference model has not been defined - please choose a reference model')
if 'param' not in locals() and 'param' not in globals():
    param = {}
    param['ThreshType'] = 'local'
    param['percentile_or_value'] = 'percentile'
    param['LocalThresholdType'] = 'minmaxmean'
    param['percentile_low'] = 25
    param['percentile_high'] = 75
#handling what to do if 'param' not provide
if 'param' not in locals() and 'param' not in globals():
    param = {}
    param['ThreshType'] = 'local'
    param['percentile_or_value'] = 'percentile'
    param['LocalThresholdType'] = 'minmaxmean'
    param['percentile_low'] = 25
    param['percentile_high'] = 75

## loading task structure based on pathway arg

In [7]:
if pathway == "secretion":
    taskStructure = scipy.io.loadmat('taskStructure_sec.mat')
elif pathway == "metabolism":
    taskStructure = scipy.io.loadmat('taskStructure.mat')
    taskInfos = taskStructure['taskStructure']
    taskInfos = [list(taskInfo) for taskInfo in taskInfos]
    taskInfos = [taskInfo[:5] for taskInfo in taskInfos]
else:
    raise ValueError('Pathway not valid')

## loading essential reactions for pathway

In [8]:
#loading list of reactions associated with task depending on model
#also needs to be changed for other stuff
if pathway==('secretion'):
    essentialRxns=scipy.io.loadmat(f'essentialRxnsbyTask_{ref}_sec.mat')
elif pathway==('metabolism'):
    essentialRxns=scipy.io.loadmat(f'essentialRxnsbyTask_{ref}.mat')
else:
    raise ValueError('Pathway not valid')

## check if genes in data are present in model, count genes not included and remove them from data

In [9]:
model=scipy.io.loadmat(ref)
model=model['model']
model['genes']=model['genes'][0]
data_genes=data['gene']
model_genes=model['genes']
model_genes=model['genes'][0][0]


ID_model = set()
gene_notInModel = set() #indicies of data['gene'] where a gene is not in the model

#print(len(model_genes))


for i in range(len(data_genes)):
    if not np.isin(data_genes[i], model_genes):
        gene_notInModel.add(i)
    else:
        tmpid = np.where(np.array(data_genes[i]) == model_genes)[0]
        ID_model.add(tmpid[0])

if not gene_notInModel:
    print('All genes provided in data are included in the reference model')
else:
    print(str(len(gene_notInModel)) + ' genes provided are not included in the reference model:')
    #print(data_genes[gene_notInModel])

# Remove genes and associated values that are not in the model
#pre_length_test1=len(data['gene'])
#pre_length_test2 = len(data['value'])

data['gene'] = [gene for i, gene in enumerate(data['gene']) if i not in gene_notInModel]
if SampleNumber == 1:
    data['value'] = [value for i, value in enumerate(data['value']) if i not in gene_notInModel]
else:
    data['value'] = np.delete(data['value'], list(gene_notInModel), axis=0)

#print(f'genes deleted:')
#print(pre_length_test1-len(data['gene']))
#print(f'values deleted:')
#print(pre_length_test2-len(data['value']))

828 genes provided are not included in the reference model:


## getting threshold value for data, making histogram, print figure (?)

In [10]:
# Get the threshold value and the histogram for the complete dataset
if SampleNumber > 1:
    linData = data['value'].reshape(-1, 1)
else:
    linData = data['value']

# Remove zeros from linData
linData = linData[linData != 0]

# Calculate the threshold value based on your conditions
if param['ThreshType'] == 'global' and param['percentile_or_value'] == 'percentile':
    print('RUN - global: percentile')
    l_global = np.percentile(np.log10(linData), param['percentile'])
    data['ths'] = 10 ** l_global
elif param['ThreshType'] == 'global' and param['percentile_or_value'] == 'value':
    print('RUN - global: value')
    data['ths'] = param['value']
elif param['ThreshType'] == 'local' and param['LocalThresholdType'] == 'mean':
    print('RUN - local: mean')
elif param['ThreshType'] == 'local' and param['LocalThresholdType'] == 'minmaxmean' and param[
    'percentile_or_value'] == 'percentile':
    print('RUN - local: minmaxmean: percentile')
    l_high = np.percentile(np.log10(linData), param['percentile_high'])
    data['ths_high'] = 10 ** l_high
    l_low = np.percentile(np.log10(linData), param['percentile_low'])
    data['ths_low'] = 10 ** l_low
elif param['ThreshType'] == 'local' and param['LocalThresholdType'] == 'minmaxmean' and param[
    'percentile_or_value'] == 'value':
    print('RUN - local: minmaxmean: value')
    data['ths_high'] = param['value_high']
    data['ths_low'] = param['value_low']
else:
    raise ValueError('No analysis triggered')

RUN - local: minmaxmean: percentile


## computing thresholds based on selected approch, storing in Gene_score, making table/dataframe

In [11]:
#Compute the threshold(s) depending on the approach used
Gene_score = np.zeros_like(data['value'], dtype=float)  # Initialize Gene_score as an array of zeros

if param['ThreshType'] == 'local':
    if param['LocalThresholdType'] == 'mean':
        # The threshold for each gene is equal to its mean value over all the samples
        threshold = np.mean(data['value'], axis=1)
    else:
        threshold = np.empty(len(data['gene']), dtype=float)
        for i in range(len(data['gene'])):
            expressionValue = data['value'][i, :]
            if np.mean(expressionValue) >= data['ths_high']:
                threshold[i] = data['ths_high']
            else:
                threshold[i] = max(np.mean(expressionValue), data['ths_low'])

    # Every single gene is associated with an expression score
    for i in range(SampleNumber):
        Gene_score[:, i] = 5.0 * np.log(1.0 + data['value'][:, i] / threshold)

elif param['ThreshType'] == 'global':
    Gene_score = 5.0 * np.log(1.0 + data['value'] / data['ths'])

# Table of gene scores to output
#pandas dataframe works for matlab table ig
geneScore = np.column_stack((data['gene'], Gene_score))
#testing code
#print(len(data['gene']))
#print(geneScore.shape)


geneScore_table = pd.DataFrame(geneScore, columns=sampNames)

## mapping expression data to the model, making expression dict, finding used gene levels

In [12]:
# Mapping of the expression data to the model
# initializing a dictionary for expression data
expression = {
    'gene': data['gene'],
    'Rxns': [],
    'gene_used': [],
    'count': []
}

minSum=False

# Load parsedGPR for the model based on 'pathway'
if pathway == "metabolism":
    parsedGPR = scipy.io.loadmat(f'parsedGPR_{ref}.mat')
else:
    parsedGPR = scipy.io.loadmat(f'parsedGPR_{ref}_sec.mat')

 # Extract the parsedGPR data from the loaded data
parsedGPR = parsedGPR['parsedGPR']

#maybe this can go earlier
expression['value']=Gene_score

#1st function call
gene_id, gene_expr = findUsedGenesLevels_all(model, expression)

## linking gene to model reactions, calling select from gpr_ all and manipulating data

In [13]:
#link the gene to the model reactions
#stub function
expressionRxns, gene_used = selectGeneFromGPR_all(model, gene_id, gene_expr, parsedGPR, minSum)
#print(expressionRxns.shape)
#print(len(gene_used))
#print(gene_used.keys())

#calculating statistics on genes used

#print(type(gene_used.iloc[6][0]))


In [14]:
expression['count']

[]

In [15]:
#iterate over samples/columns (should be 3)
for zz in range(SampleNumber):
    #re-initializing gene_all list for every sample
    gene_all=[] #used to store gene IDs
    #loop thru rows in geneUsed
    
    
    #want floats in list
    for j in range(len(gene_used)): #should only deal with column zz
        #check if cell in gene_used at j,zz is not empty
        #if it's not empty, there's a gene associated with the reaction in current sample
        if gene_used.iloc[j][zz]>=0: #this deviates a bit from matlab, double check
            #extracts geneID (1st element of cell), appends to gene_all
            #it seems like theres only one thing in the cell idk
            gene_all.append(gene_used.iloc[j][zz])#indexing may cause issue
    #use function to count how many times each unique ID appears in gene_all
    #gets count of how many reactions each gene is associated with in current sample
    gene_all_series=pd.Series(gene_all)
    countGene=gene_all_series.value_counts().reset_index()
    countGene.columns=['Value','Count']
    countGene['Percent']=(countGene['Count']/len(gene_all_series))*100
    
    #initialize count list to store counts of gene associations
    count=[]
    #going thru the rows in gene_used again for column zz
    for k in range(len(gene_used)):
        #same thing ig, check if theres a gene associated with the reaction
        if gene_used.iloc[k][zz]>=0:
            #get count of association for specific gene
            gene_value = float(gene_used.iat[k, zz])
            tmp = countGene[countGene['Value']==gene_value]
            tmp=  tmp['Count'].item()
            count.append(tmp)
        else:
            #set count to 0 for no associated genes
            count.append(0)
    print(len(count))
    #i guess i don't need to transpose it like in the matlab code?
    expression['count'].append(count)
    

expression['Rxns']=expressionRxns
expression['gene_used']=gene_used

C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if gene_used.iloc[j][zz]>=0: #this deviates a bit from matlab, double check
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_all.append(gene_used.iloc[j][zz])#indexing may cause issue
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

7785


C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if gene_used.iloc[j][zz]>=0: #this deviates a bit from matlab, double check
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_all.append(gene_used.iloc[j][zz])#indexing may cause issue
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

7785


C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if gene_used.iloc[j][zz]>=0: #this deviates a bit from matlab, double check
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_all.append(gene_used.iloc[j][zz])#indexing may cause issue
C:\Users\broba\AppData\Local\Temp\ipykernel_45004\2841147502.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

7785


In [16]:
expression['count']=np.array(expression['count'])

In [17]:
len(expression['count'][0])

7785

## Cleaning essential reactions

In [18]:
type(essentialRxns)

dict

In [19]:
essentialRxns

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu May 13 13:18:07 2021',
 '__version__': '1.0',
 '__globals__': [],
 'essentialRxns': array([[array([[array(['NADH2_u10m'], dtype='<U10')]], dtype=object),
         array([[array(['SUCD1m'], dtype='<U6')]], dtype=object),
         array([[array(['PDHm'], dtype='<U4')]], dtype=object),
         array([[array(['ACONTm'], dtype='<U6')],
                [array(['AKGDm'], dtype='<U5')],
                [array(['CSm'], dtype='<U3')],
                [array(['FUMm'], dtype='<U4')],
                [array(['ICDHxm'], dtype='<U6')],
                [array(['MDHm'], dtype='<U4')],
                [array(['SUCD1m'], dtype='<U6')],
                [array(['SUCOAS1m'], dtype='<U8')]], dtype=object),
         array([[array(['ACONTm'], dtype='<U6')],
                [array(['AKGDm'], dtype='<U5')],
                [array(['ATPS4m'], dtype='<U6')],
                [array(['ATPtm'], dtype='<U5')],
                [array(['CSm'], dty

- rxnID would be longer than one with a length 1 reaction list if that rxn shows up multiple times in model.rxns, just return first id in that case
- I think rxn id gives you an index in the model where the current rxn is
- will need to clean essentialRxns
- essentialRxns is magically ok ig

In [20]:
list_test=556
list_test=[list_test]

In [21]:
list_test

[556]

## Defining findRxnIDs function

In [22]:
def findRxnIDs(model,rxnList):
    #MATLAB DOESN'T HAVE 0 index-- needs fix
    #may need to change this so you get int/list of int
    
    
    #1 - check if rxnList is cell
    #2 - if it is an array, call ismember returning logical array
    #-----same size as rxnList with 1 or zero if the element is in model['rxns']
    # 3 return logical array(tmp) and indicies as rxnID
    

    if len(rxnList)>1:
        tmp=np.isin(rxnList,model)# may need to clean model
        #I think this'll work but maybe the types are screwy
        rxnID_tmp=[]
        for i in range(len(tmp)):
            if tmp[i]==True:
                rxnStr=rxnList[i][0][0]
                rxnID_tmp.append(np.where(model==rxnList[i])) 
            else:
                rxnID_tmp.append([]) #might cause issue, seeing 0 as index
        
        #making rxn into a list of integers for indexing(theres a better way to do this probably)
        rxnID=[]
        for i in range(len(rxnID_tmp)):
            try:
                rxnID.append(rxnID_tmp[i][0][0])
            
            except IndexError:
                rxnID.append([]) #there's definately a less dumb way to do this
                
            
            
                
    
    #4 excecutes next bit if rxnList is not a cell
    #-----assuming this means that there's only one element not 100%
    #5 calls find(strcmp), getting list of indicies of where your string shows up in the array
    #6 if this is an empty list, set rxnID to 0, return
    #7 if list is longer than one, return the first element as rxnID
    #8 if its just one thing, return the index
    else:
        rxnList=rxnList[0][0][0]
        rxnID_tmp=np.where(model==rxnList)[0] #might need to deal with types
        if len(rxnID_tmp)==0:
            rxnID_tmp=[]
        if len(rxnID_tmp)>1:
            rxnID_tmp=rxnID_tmp[0]
            
        #getting index as integer (or empty list)
        try:
            rxnID=rxnID_tmp[0]
            rxnID=[rxnID]
            #making rxnID into a list with a single int (for loop below)
            #might be going in circles here, I'll come back to it.
            
            
        except IndexError:
            rxnID=[] #there's definately a less dumb way to do this
        
        
    
    #might need to test functionality for returning empty lists
    #might need this to return as an int or list of int
    return rxnID #this is gonna be a list of indicies if the first bit excecutes or a single one

## testing findRxnIDs

## cleaning essentialRxns/ model reactions

In [23]:
essentialRxns=essentialRxns['essentialRxns']

In [24]:
essentialRxns=essentialRxns[0]

In [25]:
model_rxns=model['rxns'][0][0]

## setting vars for scores

In [79]:
#defining variables for score computation

#not sure what expressionRxns is and gpt doesn't seem to know either(says its names of rxns)
#it's all floats so maybe some sort of score?
expressionRxns=expression['Rxns'] #i think this is redundant??


#significance values used to weigh expression levels of essential reactions
#1/ the number of the genes associated with the expression
#higher number of associated genes means lower count??
significance=1/expression['count']
significance=significance.T


#i think these next 2 handle dividing by 0 in matlab, maybe unnecessary 
significance[np.isinf(significance)] = 0
significance[np.isneginf(significance)] = 0
ScorebyTask=[None]*len(taskInfos)
ScorebyTask_binary=np.full((len(taskInfos),SampleNumber), None, dtype=object)

C:\Users\broba\AppData\Local\Temp\ipykernel_45004\3679868055.py:11: RuntimeWarning: divide by zero encountered in divide
  significance=1/expression['count']


## Testing for score computation

In [27]:
type(significance[10])

numpy.ndarray

In [28]:
significance[10]

array([0.33333333, 0.5       , 0.33333333])

In [29]:
significance.shape

(7785, 3)

In [30]:
type(significance[10])

numpy.ndarray

## testing expvalue/df.drop()

what you're trying to fix --KeyError: '[2, 19, 26] not found in axis'
- expValue.drop(indicies,axis=0) has key error
- where are indicies set
- double check matlab code, make sure you're doing it right
- its the indicies being preserved from the original df, might want to reset them but i'm not sure if that'll mess me up
    - pretty sure this is fine actually

In [31]:
#1st rxnID value to have one or more empty rows in expValue/expressionRxns
rxnID_tmp=[122, 192, 4109, 4110, 396, 6099, 4235, 591, 3887, 684, 729, 831, 
           4556, 1051, 1097, 1098, 4643, 1178, 4688, 4730, 1493, 1509, 1511, 
           1515, 1583, 1587, 6314, 4828, 1863, 1865, 1901]

In [32]:
expValue_tmp=expressionRxns.loc[rxnID_tmp].reset_index(drop=True)

In [33]:
type(expValue_tmp)

pandas.core.frame.DataFrame

In [34]:
#preserved indices from expressionRxns may be the issue 
expValue_tmp

,expressionRxns1,expressionRxns2,expressionRxns3
0,6.385277,4.745314,4.083696
1,3.712999,3.950867,2.637912
2,-1.000000,-1.000000,-1.000000
3,9.096137,8.576847,9.592291
4,7.255330,5.974352,5.044748
5,0.000000,0.000000,0.000000
6,4.528078,5.650344,3.721875
7,11.905825,10.270561,13.889071
8,7.700270,6.790281,3.906134
9,13.952333,13.886678,13.552533


In [35]:
np.sum(expValue_tmp, axis=1)

0     15.214288
1     10.301778
2     -3.000000
3     27.265274
4     18.274430
5      0.000000
6     13.900297
7     36.065457
8     18.396685
9     41.391544
10    12.905675
11    41.157923
12     1.264027
13    11.367353
14    38.033411
15    38.033411
16    10.378061
17    27.381722
18     2.261213
19    -3.000000
20     9.598952
21    25.248775
22    33.403141
23    19.318400
24    20.281849
25    14.441545
26    -3.000000
27    10.378061
28    15.754001
29    10.662459
30    25.186099
dtype: float64

In [36]:
#want to drop rows in expValue/signValue that sum to -SampleNumber
#using np.sum/np.where to find indices of empty rows

row_sums = np.sum(expValue_tmp, axis=1)  # Compute sum of each row in expValue
indices = np.where(row_sums == -SampleNumber)[0]


In [37]:
indices

array([ 2, 19, 26], dtype=int64)

In [38]:
expValue_tmp.iloc[2]

expressionRxns1   -1.0
expressionRxns2   -1.0
expressionRxns3   -1.0
Name: 2, dtype: float64

In [39]:
expValue_tmp.iloc[indices]

,expressionRxns1,expressionRxns2,expressionRxns3
2,-1.0,-1.0,-1.0
19,-1.0,-1.0,-1.0
26,-1.0,-1.0,-1.0


In [40]:
expValue_tmp=expValue_tmp.drop(indices)

In [41]:
expValue_tmp

,expressionRxns1,expressionRxns2,expressionRxns3
0,6.385277,4.745314,4.083696
1,3.712999,3.950867,2.637912
3,9.096137,8.576847,9.592291
4,7.255330,5.974352,5.044748
5,0.000000,0.000000,0.000000
6,4.528078,5.650344,3.721875
7,11.905825,10.270561,13.889071
8,7.700270,6.790281,3.906134
9,13.952333,13.886678,13.552533
10,3.886822,5.324049,3.694804


## testing weird if statement/ rxnID
- try recreating 1st 3 iterations, see what goes wrong

In [42]:
test_rxnID=4688
if test_rxnID!=[]:
    print('guh')

guh


In [43]:
rxn_test=rxns=essentialRxns[0]
rxnID_test=findRxnIDs(model_rxns,rxn_test)

In [44]:
rxn_test

array([[array(['NADH2_u10m'], dtype='<U10')]], dtype=object)

In [45]:
rxnID_test

[4688]

In [46]:
if rxnID_test!=[]:
    print('a')

else:
    print('b')
    

a


In [47]:
#gutted test verion of loop
#RE-RUN INITIALIZATION CELL BEFORE RE-RUNNING THIS

'''
#looping through 195 indicies of taskInfos
for i in range(len(taskInfos)):
    #checking if theres an associated rxn for index
    if len(essentialRxns[i])>0: #I guess the indicies line up
        #finding each essential rxn's index in the model
        rxns=essentialRxns[i]
        rxnID=findRxnIDs(model_rxns,rxns)#this returns a weird array thing, need int(s)
        
        
        #THIS IF STATEMENT IS MESSING UP
        #print(i)
        #print(rxnID)
        if len(rxnID)>0: #checking if the rxn is in the model
            #print(f'i={i}')
            #print('rxnID!=[]')
            #print(f'rxnID={rxnID}')
            
            
            expValue=expressionRxns.loc[rxnID].reset_index(drop=True)#rxnID may be list
            #making sure expValue will be a df for future
            if isinstance(expValue,pd.Series):
                expValue=pd.DataFrame([expValue])
                #print('originally series:')
                #print(type(expValue))
            
            signValue=significance[rxnID] #may need to do the same with this
            
            
            #if no gene is associated with a reaction, remove reation from count
            if (expValue.sum(axis=1)==-SampleNumber).any():
                #if condition is True, rows in sgnValue corresponding to expValue are removed
                row_sums = np.sum(expValue, axis=1)  # Compute sum of each row in expValue
                indices = np.where(row_sums == -SampleNumber)[0]  # Find indices where sum is equal to -SampleNumber
                
                signValue = np.delete(signValue, indices, axis=0) #deleting rows by index
                #expValue = np.delete(expValue, indices, axis=0)
                #get this to do the same as np.delete
                expValue=expValue.drop(indices,axis=0)
                
                
            #checking if expValue is empty
            #should be false if rxn ID is an empty list/ list of empty lists
            #feels weird having these 2 if statements, might need elif or smth
            
            if len(expValue)>0:
                #updating score lists if expValue df has multiple rows
                if len(expValue)>1:
                    #takes averages if expValue has more than 1 row
                    #gpt line might be wrong
                    
                    #element-wise multiplication?
                    ScorebyTask[i]=((expValue.values * signValue).sum(axis=0))/expValue.shape[0]
                    Val=expValue.sum(axis=0)/expValue.shape[0] # .values?
                    #returning indicies where val is above a threshold
                    ID_up=np.where(Val>=(5*math.log(2)))[0]
                    #setting binary scores
                    ScorebyTask_binary[i]=[0]*SampleNumber
                    ScorebyTask_binary[i][ID_up]=1
                
                #updating score lists if expValue df is just 1 row
                elif len(expValue)==1:
                    ScorebyTask[i]=expValue.values * signValue
                    ID_up=np.where(Val>=(5*math.log(2)))[0]
                    ScorebyTask_binary[i]=[0]*SampleNumber
                    ScorebyTask_binary[i][ID_up]=1
                
            #updating score lists with -1's if expValue df is empty   
            else:
                #print('inner:')
                #print(i)
                ScorebyTask[i]=[-1]*SampleNumber
                ScorebyTask_binary[i]=[-1]*SampleNumber
            
        
        #updating score lists with -1's if rxnID is empty
        #this middle else statement is excecuted when it shouldn't
        else:
            print(f'i={i}')
            print('rxnID==[]')
            print(f'rxnID={rxnID}')
            #ScorebyTask[i]=[-1]*SampleNumber
            #ScorebyTask_binary[i]=[-1]*SampleNumber  
   
    #updating score lists with -1's if essentialRxns[i] is empty
    else:
        x=2
        #print('outer:')
        #print(i)
        #ScorebyTask[i]=[-1]*SampleNumber
        #ScorebyTask_binary[i]=[-1]*SampleNumber

'''

"\n#looping through 195 indicies of taskInfos\nfor i in range(len(taskInfos)):\n    #checking if theres an associated rxn for index\n    if len(essentialRxns[i])>0: #I guess the indicies line up\n        #finding each essential rxn's index in the model\n        rxns=essentialRxns[i]\n        rxnID=findRxnIDs(model_rxns,rxns)#this returns a weird array thing, need int(s)\n        \n        \n        #THIS IF STATEMENT IS MESSING UP\n        #print(i)\n        #print(rxnID)\n        if len(rxnID)>0: #checking if the rxn is in the model\n            #print(f'i={i}')\n            #print('rxnID!=[]')\n            #print(f'rxnID={rxnID}')\n            \n            \n            expValue=expressionRxns.loc[rxnID].reset_index(drop=True)#rxnID may be list\n            #making sure expValue will be a df for future\n            if isinstance(expValue,pd.Series):\n                expValue=pd.DataFrame([expValue])\n                #print('originally series:')\n                #print(type(expValue)

In [48]:
#seems to fix the problem, I guess there are no empty lists for rxnID with this data

## computing scores

notes:
- may need further debugging
- expressionRxns comes from selectGeneFromGPR stub, they're floats but I'm not sure what they do
- if this gives the wrong output, may need to make 'if rxnID!=[]:'  handle list of empty lists
- I think this is where you get all the output of the quickstart, maybe table the detailscoring and do the stub first

In [49]:
import math

In [74]:
#for testing purposes
binary_expected=np.genfromtxt('score_binary_expected.csv',delimiter=',')

In [76]:
binary_expected[5]==ScorebyTask_binary[5]

array([ True,  True,  True])

In [50]:
#suppress warnings for now
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [80]:
#RE-RUN INITIALIZATION CELL BEFORE RE-RUNNING THIS

#looping through 195 indicies of taskInfos
for i in range(len(taskInfos)):
    #checking if theres an associated rxn for index
    if len(essentialRxns[i])>0: #I guess the indicies line up
        #finding each essential rxn's index in the model
        rxns=essentialRxns[i]
        rxnID=findRxnIDs(model_rxns,rxns)#this returns a weird array thing, need int(s)
        
        
        #THIS IF STATEMENT IS MESSING UP
        #print(i)
        #print(rxnID)
        if rxnID!=[]: #checking if the rxn is in the model
            #print(f'i={i}')
            #print('rxnID!=[]')
            #print(f'rxnID={rxnID}')
            
            expValue=expressionRxns.loc[rxnID].reset_index(drop=True)#rxnID may be list
            #making sure expValue will be a df for future
            if isinstance(expValue,pd.Series):
                expValue=pd.DataFrame([expValue])
                #print('originally series:')
                #print(type(expValue))
            
            signValue=significance[rxnID] #may need to do the same with this
            
            #if no gene is associated with a reaction, remove reation from count
            if (expValue.sum(axis=1)==-SampleNumber).any():
                #if condition is True, rows in sgnValue corresponding to expValue are removed
                row_sums = np.sum(expValue, axis=1)  # Compute sum of each row in expValue
                indices = np.where(row_sums == -SampleNumber)[0]  # Find indices where sum is equal to -SampleNumber
                
                signValue = np.delete(signValue, indices, axis=0) #deleting rows by index
                #expValue = np.delete(expValue, indices, axis=0)
                #get this to do the same as np.delete
                expValue=expValue.drop(indices,axis=0)
                
                
            #checking if expValue is empty
            #should be false if rxn ID is an empty list/ list of empty lists
            #feels weird having these 2 if statements, might need elif or smth
            if len(expValue)>0:
                #updating score lists if expValue df has multiple rows
                if len(expValue)>1:
                    #takes averages if expValue has more than 1 row
                    #gpt line might be wrong
                    
                    #element-wise multiplication?
                    ScorebyTask[i]=((expValue.values * signValue).sum(axis=0))/expValue.shape[0]
                    Val=expValue.sum(axis=0)/expValue.shape[0] # .values?
                    #returning indicies where val is above a threshold
                    ID_up=np.where(Val>=(5*math.log(2)))[0]
                    
                    #setting binary scores
                    #binary scores set here are good
                    ScorebyTask_binary[i]=[0]*SampleNumber
                    ScorebyTask_binary[i][ID_up]=1
                    #print(f'ID_up={ID_up}')
                    #print(f'row {i+1}:{ScorebyTask_binary[i]}')
                    #print(f'match with expected? {binary_expected[i]==ScorebyTask_binary[i]}')
                
                #updating score lists if expValue df is just 1 row
                
                #this block has issues with the binary scores
                elif len(expValue)==1:
                    ScorebyTask[i]=expValue.values * signValue
                    
                    
                    ID_up=np.where(expValue>=(5*math.log(2)))[0]
                    ScorebyTask_binary[i]=[0]*SampleNumber
                    ScorebyTask_binary[i][ID_up]=1
                    
                    #issues here
                    print(f'ID_up={ID_up}')
                    print(f'row {i+1}:{ScorebyTask_binary[i]}')
                    print(f'match with expected? {binary_expected[i]==ScorebyTask_binary[i]}')

                
            #updating score lists with -1's if expValue df is empty   
            else:
                #print('inner:')
                #print(i)
                ScorebyTask[i]=[-1]*SampleNumber
                ScorebyTask_binary[i]=[-1]*SampleNumber
        
        #updating score lists with -1's if rxnID is empty
        #this middle else statement is excecuted when it shouldn't
        else:
            #print(f'i={i}')
            #print('rxnID!=[]')
            #print(f'rxnID={rxnID}')
            #print('middle:')
            #print(i)
            #print(rxnID)
            ScorebyTask[i]=[-1]*SampleNumber
            ScorebyTask_binary[i]=[-1]*SampleNumber  
   
    #updating score lists with -1's if essentialRxns[i] is empty
    else:
        #print('outer:')
        #print(i)
        ScorebyTask[i]=[-1]*SampleNumber
        ScorebyTask_binary[i]=[-1]*SampleNumber

ID_up=[]
row 1:[0 0 0]
match with expected? [ True  True  True]
ID_up=[0 0 0]
row 2:[1 0 0]
match with expected? [ True False False]
ID_up=[0]
row 3:[1 0 0]
match with expected? [ True  True  True]
ID_up=[0 0 0]
row 7:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0 0]
row 8:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0 0]
row 18:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0]
row 19:[1 0 0]
match with expected? [False False False]
ID_up=[0 0]
row 20:[1 0 0]
match with expected? [False False False]
ID_up=[0 0 0]
row 31:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0]
row 33:[1 0 0]
match with expected? [False False False]
ID_up=[0 0 0]
row 34:[1 0 0]
match with expected? [ True False False]
ID_up=[0]
row 36:[1 0 0]
match with expected? [ True  True  True]
ID_up=[0 0 0]
row 39:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0 0]
row 56:[1 0 0]
match with expected? [ True False False]
ID_up=[0 0 0]
row 57:[1 0 0]
match with expe

In [54]:
ScorebyTask

[array([[2.25898881e+00, 9.19038303e-04, 1.30515123e-03]]),
 array([[5.50012484, 6.07998477, 4.17389106]]),
 array([[1.2376663 , 1.62402073, 0.87930397]]),
 array([3.09184485, 3.15732991, 2.4147695 ]),
 array([3.79722342, 3.95604914, 3.8219504 ]),
 array([8.01637232, 8.53457222, 8.85601948]),
 array([[2.65669034, 1.96777029, 1.33119316]]),
 array([[1.80155479, 2.32709064, 1.66810684]]),
 array([2.19345452, 2.46334372, 2.60718755]),
 array([1.91391205, 1.91963868, 2.07230701]),
 array([2.28408932, 2.52660543, 2.64747968]),
 array([1.93533627, 1.90864546, 2.07926916]),
 array([1.98889531, 2.21685641, 2.34593847]),
 array([1.80741343, 1.80350291, 1.97131362]),
 array([2.39114617, 2.64283043, 2.77964291]),
 array([2.05674167, 2.02203699, 2.21855589]),
 array([1.70582131, 1.67657497, 2.12877606]),
 array([[2.27367404, 2.69897541, 2.95989402]]),
 array([[1.14605221, 2.05718205, 1.90364199]]),
 array([[1.14605221, 2.05718205, 1.90364199]]),
 array([0.87023594, 2.02019076, 1.09371548]),
 array

In [55]:
len(ScorebyTask)

195

In [56]:
len(ScorebyTask_binary)

195

In [81]:
ScorebyTask_binary

array([[0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1,

In [82]:
ScorebyTask_binary[18]

array([1, 0, 0], dtype=object)

In [83]:
binary_expected[18]

array([0., 1., 1.])

In [ ]:
taskInfos[2]

## output / debug 

taskInfos:
- taskInfos seems like its got the right stuff in there, just messy
- seems like taskInfos is just unchanged data loaded from one of the .mat files, not sure how this is considered output

score/binary score:
- iterations w/ single int was original problem
- normal scores is correct at first glance
- score binary wrong
- appears that maybe first column is correct, check on this next
- scoreByTask is a list of lists with none or array, should be array w/ 3 cols, 195 rows - probably not main issue but needed to write files

score binary issue:
- wrong entries
- 1st column correct - not actually but I swear it was
- other 2 both wrong but look random
- I think ID_up is the problem
- binary scores set in 'if len(expValue)>1:' seem good
- could be initializations somehow
- source of issues in 'elif len(expValue)==1:'
- why is the first column causing problems when it looked like it was good??


In [60]:
binary_expected=np.genfromtxt('score_binary_expected.csv',delimiter=',')

In [61]:
binary_expected

array([[ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  1.,  1.],
       [ 0.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  1.

In [63]:
binary_expected[:,0]

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,
        1.,  0.,  0.,  1., -1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
        0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1

In [64]:
ScorebyTask_binary[:,0]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, -1, -1, 0, 1, 0, 1, 1, 1, 1, 1],
      dtype=object)

In [65]:
binary_expected[0][0]

0.0

In [66]:
ScorebyTask_binary[0][0]

0

In [67]:
binary_expected[0][0]==ScorebyTask_binary[0][0]

True

In [68]:
binary_expected[:,0]==binary_expected[:,0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [70]:
len(ScorebyTask_binary[:,0])

195